# Principal component analysis (PCA)

In [ ]:
%%capture
! pip install scikit-learn 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [12]:
raw_data = "../data/processed.csv"
dataset = pd.read_csv(raw_data)
dataset.head()

,ckpt_name,training_hyperparams.cosine_final_lr_ratio,training_hyperparams.optimizer,training_hyperparams.batch_accumulate,training_hyperparams.phase_callbacks,training_hyperparams.initial_lr.backbone,training_hyperparams.initial_lr.default,training_hyperparams.schema.properties.initial_lr.anyOf,training_hyperparams.max_epochs,dataset_params.train_dataset_params,...,Precision@0.50:0.95,Recall@0.50:0.95,mAP@0.50:0.95,F1@0.50:0.95,AP@0.50:0.95_Mass,AP@0.50:0.95_Calcification,Best_score_threshold,Best_score_threshold_Mass,Best_score_threshold_Calcification,Target
0,RUN_20240612_100027_359642,0.1,Adam,1,['<super_gradients.training.utils.callbacks.ca...,0.10,0.10,"[{'type': ['number', 'string', 'boolean', 'nul...",50,"{'data_dir': '../data/', 'images_dir': 'train/...",...,0.000106,0.005259,0.000007,0.000182,0.000010,0.000004,0.11,0.11,0.22,0
1,RUN_20240617_163510_293224,0.1,Adam,1,['<super_gradients.training.utils.callbacks.ca...,0.10,0.10,"[{'type': ['number', 'string', 'boolean', 'nul...",50,"{'data_dir': '../data/', 'images_dir': 'train/...",...,0.000132,0.011994,0.000007,0.000235,0.000006,0.000009,0.11,0.11,0.22,0
2,RUN_20240620_104658_182467,0.1,AdamW,1,['<super_gradients.training.utils.callbacks.ca...,0.10,0.10,"[{'type': ['number', 'string', 'boolean', 'nul...",75,"{'data_dir': '../data/', 'images_dir': 'train/...",...,0.000715,0.076085,0.005795,0.001406,0.000080,0.011509,0.25,0.10,0.25,0
3,RUN_20240624_175224_278149,0.1,AdamW,1,['<super_gradients.training.utils.callbacks.ca...,0.01,0.01,"[{'type': ['number', 'string', 'boolean', 'nul...",75,"{'data_dir': '../data/', 'images_dir': 'train/...",...,0.001235,0.119091,0.011001,0.002431,0.000340,0.021661,0.25,0.18,0.25,0
4,RUN_20240625_113055_125920,0.1,AdamW,1,['<super_gradients.training.utils.callbacks.ca...,0.01,0.01,"[{'type': ['number', 'string', 'boolean', 'nul...",100,"{'data_dir': '../data/', 'images_dir': 'train/...",...,0.001634,0.147835,0.015902,0.003215,0.000385,0.031419,0.25,0.18,0.25,0


### Drop label columns

Since PCA is unsupervised, we don't want non-feature columns influencing the results

In [14]:
non_features = ['ckpt_name', 'Precision@0.50:0.95', 'Recall@0.50:0.95', 'mAP@0.50:0.95', 'F1@0.50:0.95', 
                'AP@0.50:0.95_Mass', 'AP@0.50:0.95_Calcification', 'Best_score_threshold',
                'Best_score_threshold_Mass', 'Best_score_threshold_Calcification', 'Target']

x = dataset.drop(columns=non_features)
y = dataset['Target'] # this is stored in variable 'y' for later (training a ML model)

x.head()

,training_hyperparams.cosine_final_lr_ratio,training_hyperparams.optimizer,training_hyperparams.batch_accumulate,training_hyperparams.phase_callbacks,training_hyperparams.initial_lr.backbone,training_hyperparams.initial_lr.default,training_hyperparams.schema.properties.initial_lr.anyOf,training_hyperparams.max_epochs,dataset_params.train_dataset_params,dataset_params.train_dataloader_params.batch_size,dataset_params.train_dataloader_params.num_workers,dataset_params.train_dataloader_params.collate_fn,dataset_params.valid_dataset_params,dataset_params.valid_dataloader_params.batch_size,dataset_params.valid_dataloader_params.num_workers,dataset_params.valid_dataloader_params.collate_fn,additional_log_items.initial_LR.backbone,additional_log_items.initial_LR.default
0,0.1,Adam,1,['<super_gradients.training.utils.callbacks.ca...,0.10,0.10,"[{'type': ['number', 'string', 'boolean', 'nul...",50,"{'data_dir': '../data/', 'images_dir': 'train/...",16,8,<super_gradients.training.utils.collate_fn.det...,"{'data_dir': '../data/', 'images_dir': 'valid/...",16,8,<super_gradients.training.utils.collate_fn.det...,0.10,0.10
1,0.1,Adam,1,['<super_gradients.training.utils.callbacks.ca...,0.10,0.10,"[{'type': ['number', 'string', 'boolean', 'nul...",50,"{'data_dir': '../data/', 'images_dir': 'train/...",16,8,<super_gradients.training.utils.collate_fn.det...,"{'data_dir': '../data/', 'images_dir': 'valid/...",16,8,<super_gradients.training.utils.collate_fn.det...,0.10,0.10
2,0.1,AdamW,1,['<super_gradients.training.utils.callbacks.ca...,0.10,0.10,"[{'type': ['number', 'string', 'boolean', 'nul...",75,"{'data_dir': '../data/', 'images_dir': 'train/...",10,5,"PPYoloECollateFN(random_resize_sizes=None, ran...","{'data_dir': '../data/', 'images_dir': 'valid/...",10,5,"PPYoloECollateFN(random_resize_sizes=None, ran...",0.10,0.10
3,0.1,AdamW,1,['<super_gradients.training.utils.callbacks.ca...,0.01,0.01,"[{'type': ['number', 'string', 'boolean', 'nul...",75,"{'data_dir': '../data/', 'images_dir': 'train/...",4,2,"PPYoloECollateFN(random_resize_sizes=None, ran...","{'data_dir': '../data/', 'images_dir': 'valid/...",4,2,"PPYoloECollateFN(random_resize_sizes=None, ran...",0.01,0.01
4,0.1,AdamW,1,['<super_gradients.training.utils.callbacks.ca...,0.01,0.01,"[{'type': ['number', 'string', 'boolean', 'nul...",100,"{'data_dir': '../data/', 'images_dir': 'train/...",4,2,"PPYoloECollateFN(random_resize_sizes=None, ran...","{'data_dir': '../data/', 'images_dir': 'valid/...",4,2,"PPYoloECollateFN(random_resize_sizes=None, ran...",0.01,0.01


In [15]:
print(x.isna().sum())

training_hyperparams.cosine_final_lr_ratio                 0
training_hyperparams.optimizer                             0
training_hyperparams.batch_accumulate                      0
training_hyperparams.phase_callbacks                       0
training_hyperparams.initial_lr.backbone                   0
training_hyperparams.initial_lr.default                    0
training_hyperparams.schema.properties.initial_lr.anyOf    0
training_hyperparams.max_epochs                            0
dataset_params.train_dataset_params                        0
dataset_params.train_dataloader_params.batch_size          0
dataset_params.train_dataloader_params.num_workers         0
dataset_params.train_dataloader_params.collate_fn          0
dataset_params.valid_dataset_params                        0
dataset_params.valid_dataloader_params.batch_size          0
dataset_params.valid_dataloader_params.num_workers         0
dataset_params.valid_dataloader_params.collate_fn          0
additional_log_items.ini

### Identify column types

In [16]:
numericals = x.select_dtypes(include=['int64', 'float64']).columns
categoricals = x.select_dtypes(include=['bool', 'object']).columns

### Preprocessing pipelines

* StandardScaler for preprocessing numerical features so each will have a mean of 0 and a stardard deviation of 1. This helps avoiding bias. 
* OneHotEncoder converts categorical features into a format that can be provided to ML algorithms to do a better job in prediction. This transformation creates a binary column for each category

In [17]:
numericals_transformer = StandardScaler()
categoricals_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers= [
        ('num', numericals_transformer, numericals),
        ('cat', categoricals_transformer, categoricals)
    ])

### Create a pipeline that includes PCA

In [26]:
pca = PCA(n_components=3) # Define PCA instance

# Chain preprocessing and PCA into a single workflow
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('pca', pca)
])

In [27]:
# Fit and transform the data using the pipeline
x_pca = pipeline.fit_transform(x)

In [29]:
pca_df = pd.DataFrame(data=x_pca, columns=['PC1', 'PC2', 'PC3'])
pca_df['Target'] = y.values
print(pca_df)

         PC1       PC2       PC3  Target
0   4.720585  1.048849 -0.332982       0
1   4.720585  1.048849 -0.332982       0
2   3.128465 -1.283198 -1.209484       0
3  -1.100292 -3.491603 -0.611232       0
4  -1.221448 -3.661834 -0.149514       0
5  -0.507832 -2.079319  1.182843       0
6   0.789280  0.790011  3.532711       0
7  -2.231871  1.348608  0.064315       1
8  -2.231871  1.348608  0.064315       0
9  -1.989559  1.689072 -0.859121       1
10 -1.844171  1.893350 -1.413183       1
11 -2.231871  1.348608  0.064315       0


### Determine the number of principal components

Explained Variance Ratio can help determine how many components to retain, since the purpose of PCA is reducing dimensions while preserving the most importnt variance in the data. 

In [30]:
pca = PCA()
x_pca = pca.fit_transform(x)

# Plot cumulative explained variance
plt.plot(range(1, len(pca.explained_variance_ratio_)+1),
         np.cumsum(pca.explained_variance_ratio_))

plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance Ratio vs. Number of Components')
plt.show()

ValueError: could not convert string to float: 'Adam'